#### ID 10302

```You’re given a dataset of uber rides with the traveling distance (‘distance_to_travel’) and cost (‘monetary_cost’) for each ride. First, find the difference between the distance-per-dollar for each date and the average distance-per-dollar for that year-month. Distance-per-dollar is defined as the distance traveled divided by the cost of the ride. Use the calculated difference on each date to calculate absolute average difference in distance-per-dollar metric on monthly basis (year-month). --The output should include the year-month (YYYY-MM) and the absolute average difference in distance-per-dollar (Absolute value to be rounded to the 2nd decimal). You should also count both success and failed request_status as the distance and cost values are populated for all ride requests. Also, assume that all dates are unique in the dataset. Order your results by earliest request date first.```

In [ ]:
%%sql
WITH distance_per_dollar AS (SELECT request_date,
                                    TO_CHAR(request_date, 'YYYY-MM')   AS request_mnth,
                                    distance_to_travel / monetary_cost AS distance_per_dollar
                             FROM uber_request_logs),
     daily_diff AS (SELECT request_mnth,
                           ABS(distance_per_dollar - AVG(distance_per_dollar)
                                                     OVER (PARTITION BY request_mnth)) AS difference
                    FROM distance_per_dollar)
select request_mnth, round(avg(difference)::decimal, 2) as difference
from daily_diff
group by request_mnth 
order by request_mnth

In [ ]:
df = uber_request_logs

df['request_mnth'] = df['request_date'].dt.strftime('%Y-%m')

df['distance_per_dollar'] = df['distance_to_travel'] / df['monetary_cost']

df['avg_distance_per_dollar'] = df.groupby('request_mnth')['distance_per_dollar'].transform('mean')

df['difference'] = (df['distance_per_dollar'] - df['avg_distance_per_dollar']).abs()

df.groupby('request_mnth').agg(abs_difference=('difference', lambda x: x.mean().round(2))).reset_index().sort_values(
    'request_mnth')